# Quiz 2 : Continuation 
## S.Bhuvanesh
## 2018113002

In [1]:
import numpy as np
import sympy
import scipy.linalg
import re #RegEx : To convert string into workable form
import math

# Solve a given chemical equation

## Task-1: Design a input scheme for the user to input a chemical equation that need to be balanced.



In [2]:
rawString = input()


KClO3 + P4 = P4O10 + KCl


In [3]:
#First finding all the unique elements in the whole equation:
print(rawString)
rawString = rawString + " "
temp=[]
elements = re.findall("[A-Z][^A-Z^\s]*",rawString)
for i in elements :
    print(i)
    '''if len(i) > 1 and (re.search("[^a-z]",i[-1])) :
        temp = np.append(temp,i[:-1])
    else :
        temp = np.append(temp,i)'''
    temp = np.append(temp,re.split('(\d+)',i)[0])
elements = np.unique(temp)

print("Elements : " , str(elements))
noEle = elements.size # Number of unique elements in the equation

#Let us assign a dimension number to each of the element present in the equation:
dicEle = {}
for i in range(noEle):
    dicEle[elements[i]] = i


#Now each molecule in our equation would be represented by a noEle dimensional vector with value equal to the number
# of atoms of that element in that molecule:

#Spliting the input String into LHS and RHS :

LHS = rawString.split('=')[0]
RHS = rawString.split('=')[1]

LHS = re.findall("[A-Za-z0-9]+",LHS)
print("LHS " + str(LHS))


RHS = re.findall("[A-Za-z0-9]+",RHS)
print("RHS " + str(RHS))

noLHS = len(LHS) #number of vectors in 
noRHS = len(RHS)

#For Each reactant or molecule , let us declare an np.array as a vector
temp = np.zeros(noEle)
LHSvects = np.array([temp])
RHSvects = np.array([temp])
for i in range(noLHS):
    tempVect = np.zeros((noEle))
    tempVect = np.array([tempVect])
    components = re.findall("[A-Z][^A-Z]*" ,LHS[i])
    for k in components:
        if k[-1].isdigit():
            tempVect[0][dicEle[re.split('(\d+)',k)[0]]] = re.split('(\d+)',k)[1]
        else:
            tempVect[0][dicEle[k]] = 1
    LHSvects = np.append(LHSvects,tempVect,axis=0)
LHSvects = LHSvects[1:]
for i in range(noRHS):
    tempVect = np.zeros(noEle)
    tempVect = np.array([tempVect])
    components = re.findall("[A-Z][^A-Z]*" ,RHS[i])
    print("Components: " + str(components))
    for k in components:
        if k[-1].isdigit():
            tempVect[0][dicEle[re.split('(\d+)',k)[0]]] = re.split('(\d+)',k)[1]
        else:
            tempVect[0][dicEle[k]] = 1
    RHSvects = np.append(RHSvects,tempVect,axis=0)
RHSvects = -RHSvects[1:]
totalEqn = np.append(LHSvects,RHSvects,axis=0)
totalEqn = np.transpose(totalEqn)
LHSvects = np.transpose(LHSvects)
RHSvects = np.transpose(RHSvects)
#print(LHSvects)
#print(RHSvects)
print(totalEqn)


KClO3 + P4 = P4O10 + KCl
K
Cl
O3
P4
P4
O10
K
Cl
Elements :  ['Cl' 'K' 'O' 'P']
LHS ['KClO3', 'P4']
RHS ['P4O10', 'KCl']
Components: ['P4', 'O10']
Components: ['K', 'Cl']
[[  1.   0.  -0.  -1.]
 [  1.   0.  -0.  -1.]
 [  3.   0. -10.  -0.]
 [  0.   4.  -4.  -0.]]


## Task-2: From the input, get into a form that is suitable for mathematical manipulations

 HINT: consider using sympy.

1. Lets say that you have atom types $A_1, A_2, \cdots A_N$. 
2. Write each chemical component as a $N$ dimensional vector 
3. Suppose the balanced the chemical reaction is where $R_i$ are reactants and $P_i$ are products, and we require that $r_i$ and $p_i$ be integers... 
$$ r_1 R_1 + r_2 R_2 +\cdots - (p_1 P_1 + p_1 P_2 ) = 0$$ 
3. Set up a system of $N$ simulatneous equations, and solve for it to get $r_i$ and $p_i$
4. What are the conditions under which there is NO solution? 

In [4]:
#Now we have converted the problem into a mathematical problem of finding a NULLSPACE of the matrix transformation
#Given by the matrix totalEqn 
soln = scipy.linalg.null_space(totalEqn)
print(soln)
#Now by solving that system of linear Equations, we have found the ratio in which the molecules are supposed to be..
# We are required to convert them in integral form
soln = soln/min(soln)
print(soln)

[[-0.67728546]
 [-0.20318564]
 [-0.20318564]
 [-0.67728546]]
[[1. ]
 [0.3]
 [0.3]
 [1. ]]


In [5]:
print(np.transpose(soln))
print(soln)
soln = np.transpose(soln)

[[1.  0.3 0.3 1. ]]
[[1. ]
 [0.3]
 [0.3]
 [1. ]]


In [9]:
#Now we try to find the smallest ratio in which every thing is integral:
def isInt(x):
    bo = True
    for i in x[0]:
        bo = bo and (math.ceil(i)-i < 0.00001 or i-math.floor(i) < 0.00001)
    return bo
for i in range(2,101):
    tempSoln = soln * i 
    if isInt(tempSoln):
        soln = tempSoln
        break

## Task-3: Using the solution above, output the balanced equation

In [ ]:
#Now We simply output the above solution as the coefficients in the string

In [10]:
output = ""
countComp = 0
for i in range(noLHS) :
    if i!= 0 :
        output += " + "
    output += (str(int(np.rint(soln[0][countComp])))+ " " + LHS[i] )
    countComp += 1
output += " = "
for i in range(noRHS) :
    if i!= 0 :
        output += " + "
    output += (str(int(np.rint(soln[0][countComp]))) + " " + RHS[i])
    countComp += 1
print(output)

10 KClO3 + 3 P4 = 3 P4O10 + 10 KCl


## Task-4: If the above is working, can you wrap the whole program in a single function, which will take the input and give the proper output?

Given the string 'eqn', the output of the function that you will write, called BalanceChemicalEquation(eqn), will be the balanced chemical equation or a proper error